```
This software is a part of GPU Ocean.

Copyright (C) 2018, 2019 SINTEF Digital

This notebook sets up a demo for handling dry states (land).

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import os
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import datetime
import sys

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
#rc('figure', figsize=(16.0, 12.0))
#rc('animation', html='html5')
plt.rcParams["animation.html"] = "jshtml"

#Import our simulator
from SWESimulators import KP07, CDKLM16, PlotHelper, Common, WindStress, IPythonMagic
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
%cuda_context_handler gpu_ctx

In [ ]:
#Create output directory for images
#imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
#os.makedirs(imgdir)
#print("Saving images to " + imgdir)

In [ ]:
# Set initial conditions common to all simulators
sim_args = {
"gpu_ctx": gpu_ctx,
"nx": 100, "ny": 200,
"dx": 200.0, "dy": 200.0,
"dt": 1,
"g": 9.81,
"f": 0.0012,
"coriolis_beta": 0,
"r": 0.0
}

In [ ]:

def sim_animation(simulator, T, substep=10.0, skips=0):
    
    const_y = 10
    eta1, u1, v1 = sim.download(interior_domain_only=True)
    simH_xsec = sim.downloadBathymetry()[1][2:-2, const_y]

    
    #Create figure and plot initial conditions
    #fig = plt.figure(figsize=(12, 15))
    
    fig = plt.figure(figsize=(12,10))

    gs = fig.add_gridspec(3,3)
    ax_eta  = fig.add_subplot(gs[0:2, 0])
    ax_u    = fig.add_subplot(gs[0:2, 1])
    ax_v    = fig.add_subplot(gs[0:2, 2])
    ax_xsec = fig.add_subplot(gs[2, :])
    
    domain_extent = [0, sim.nx*sim.dx, 0, sim.ny*sim.dy]
    
    #ax_eta = plt.subplot(1,3,1)
    #ax_eta = plt.subplot(2,3,1)
    sp_eta = ax_eta.imshow(eta1, interpolation="none", origin='bottom', vmin=-1.5, vmax=1.5, extent=domain_extent)
    
    #ax_u = plt.subplot(1,3,2)
    #ax_u = plt.subplot(2,3,2)
    sp_u = ax_u.imshow(u1, interpolation="none", origin='bottom', vmin=-1.5, vmax=1.5, extent=domain_extent)
    
    #ax_v = plt.subplot(1,3,3)
    #ax_v = plt.subplot(2,3,3)
    sp_v = ax_v.imshow(v1, interpolation="none", origin='bottom', vmin=-1.5, vmax=1.5, extent=domain_extent)
    
    #ax_xsec = plt.subplot(2,3,4)
    sp_etaxsec, = ax_xsec.plot(range(sim.ny), eta1[:, const_y], color='xkcd:ocean')
    sp_zero,    = ax_xsec.plot(range(sim.ny), np.zeros_like(eta1[:,10]), color='xkcd:ocean', linestyle='dashed')
    sp_Hconst,  = ax_xsec.plot(range(sim.ny), -simH_xsec, color='xkcd:brown', zorder=10)
    ax_xsec.set_ylabel('eta at x = 100')
    ax_xsec.set_xlabel('y cell')
    
    if skips > 0:
        sim.step(skips*substep)
    
    #Helper function which simulates and plots the solution
    def animate(i):
        if (i>0):
            t = sim.step(substep)
        else:
            t = 0.0
        eta1, u1, v1 = sim.download(interior_domain_only=True)
        
        #Update plots
        fig.sca(ax_eta)
        sp_eta.set_data(eta1)
        
        fig.sca(ax_u)
        sp_u.set_data(u1)
        
        fig.sca(ax_v)
        sp_v.set_data(v1)
        
        #fig.sca(ax_xsec)
        xsec = np.maximum(eta1[:, const_y], -simH_xsec)
        sp_etaxsec.set_ydata(xsec)
        
        fig.suptitle("Time = {:04.0f} s ({:s})".format(t, sim.__class__.__name__), fontsize=18)
        #print("."+str(np.sum(eta0))+".", end='')
        print(".", end='')
        

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    
    return anim

In [ ]:
def linearCase(low, high, use_cdklm=True, T=60, substep=100):
    ghosts = np.array([2,2,2,2]) # north, east, south, west
    dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
                 sim_args["nx"] + ghosts[1]+ghosts[3])

    H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 60.0
    eta0 = np.zeros(dataShape, dtype=np.float32)
    u0 = np.zeros(dataShape, dtype=np.float32)
    v0 = np.zeros(dataShape, dtype=np.float32)

    # Create sloping bathymetry
    linearBathymetryY(H, sim_args['nx'], sim_args['ny'], sim_args['dx'], sim_args['dy'], ghosts, high, low)
    #plt.imshow(H, origin='lower', interpolation='none')
    #plt.colorbar()
    #print('H.shape: ', H.shape)

    #Create dam break in to south of domain
    addStraightDam(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], ghosts, 3)

    #fig = plt.figure()
    #plt.imshow(eta0, origin='lower', interpolation='none')
    #plt.colorbar()


    #Initialize simulator
    fvm_args = {"H": H, "eta0": eta0, "hu0": u0, "hv0": v0,
                "boundary_conditions": Common.BoundaryConditions(1,2,1,2)}

    sim = None
    if use_cdklm:
        sim = CDKLM16.CDKLM16(**fvm_args, **sim_args)
    else:
        sim = KP07.KP07(**fvm_args, **sim_args)

    #sim_H = sim.downloadBathymetry()
    #print('sim_H.shape: ', sim_H[0].shape)
    #fig = plt.figure()
    #plt.imshow(sim_H[0], origin='lower', interpolation='none')
    #plt.colorbar()


    return sim
    #Run a simulation and plot it
    #return sim_animation(sim, T=T, substep=substep)

In [ ]:
sim = linearCase(0.5, 10, use_cdklm=False)
sim_animation(sim, T=10, substep=189, skips=50)

In [ ]:
ghosts = np.array([2,2,2,2]) # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 60.0
eta0 = np.zeros(dataShape, dtype=np.float32)
u0 = np.zeros(dataShape, dtype=np.float32)
v0 = np.zeros(dataShape, dtype=np.float32)

# Create sloping bathymetry
linearBathymetryY(H, sim_args['nx'], sim_args['ny'], sim_args['dx'], sim_args['dy'], ghosts, 10, -5)
#plt.imshow(H, origin='lower', interpolation='none')
#plt.colorbar()
#print('H.shape: ', H.shape)

#Create dam break in to south of domain
addStraightDam(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], ghosts, 3)

#fig = plt.figure()
#plt.imshow(eta0, origin='lower', interpolation='none')
#plt.colorbar()


#Initialize simulator
ctcs_args = {"H": H, "eta0": eta0, "hu0": u0, "hv0": v0, "use_rk2": True,
            "boundary_conditions": Common.BoundaryConditions(1,2,1,2)}
sim = KP07.KP07(**ctcs_args, **sim_args)

#sim_H = sim.downloadBathymetry()
#print('sim_H.shape: ', sim_H[0].shape)
#fig = plt.figure()
#plt.imshow(sim_H[0], origin='lower', interpolation='none')
#plt.colorbar()



#Run a simulation and plot it
sim_animation(sim, T=60, substep=100)

In [ ]:
ghosts = np.array([2,2,2,2]) # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 60.0
eta0 = np.zeros(dataShape, dtype=np.float32)
u0 = np.zeros(dataShape, dtype=np.float32)
v0 = np.zeros(dataShape, dtype=np.float32)

#Create bump in to lower left of domain for testing
addCentralBump(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], ghosts)

#Initialize simulator
ctcs_args = {"H": H, "eta0": eta0, "hu0": u0, "hv0": v0, "rk_order": 2}
sim = CDKLM16.CDKLM16(**ctcs_args, **sim_args)

#Run a simulation and plot it
sim_animation(sim, T=50)

In [ ]:
str(cuda.Context.get_cache_config())